In [ ]:
!pip install qiskit qiskit-ibm-runtime pylatexenc qiskit_aer

In [ ]:
from numpy import pi
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
# 3-qubit f(100)=1 의 그로버 오라클 회로를 구현하는 함수
def grover_oracle_circuit():
    n = 3
    qy = QuantumRegister(1, 'y')
    qx = QuantumRegister(n, 'x')
    qc = QuantumCircuit(qy, qx) # qc 라는 이름의 n+1큐비트 양자 회로 객체를 생성
    # implement |x,y> ->|x, y XOR f(X)>
    # where f(100) = 1, f(x) = 0 otherwise
    qc.x([qx[1],qx[0]])      # qx[1], qx[0] 큐비트의 상태를 반전
    qc.mcx(list(qx),qy)      # 제어 큐비트 qx[0,1,2], 타겟 큐비트 qy 인 MCX gate
    # Uf|x,y>=|x,y XOR f(x)가 되도록 qx 를 uncomputation
    qc.x([qx[1],qx[0]])      # qx[1], qx[0] 큐비트의 상태를 반전하여 원상태로 복귀
    qc.name = 'Oracle'
    return qc                # qc 객체를 반환

circuit = grover_oracle_circuit() # 함수를 호출하여 회로를 표현하는 객체를 생성
circuit.draw(output='mpl') # circuit 회로를 matplot library 형식의 그림 화일로 출력

In [ ]:
# diffusion circuit for n qubits
def diffuser_circuit(n):
    x = QuantumRegister(n,'x')
    qc = QuantumCircuit(x) # n 큐비트 회로 객체 생성
    qc.h(x)                # 모든 큐비트에 H
    qc.x(x)                # 모든 큐비트에 X

    qc.barrier()
    qc.h(x[0])             # x0 (target 큐비트) 에만 H
    qc.mcx(x[1:n], x[0])   # 제어 큐비트 x[1,2,...,n-1], 타겟 큐비트 x[0] 인 다중제어 X gate
    qc.h(x[0])             # x0 (target 큐비트) 에만 H
    qc.barrier()           # HXH=Z 이기 때문에 이 결과는 다중제어-Z gate 와 같다.

    qc.x(x)                # 모든 큐비트에 X
    qc.h(x)                # 모든 큐비트에 H

    qc.name = "Diffuser"
    return qc

circuit = diffuser_circuit(3) # 함수를 호출하여 회로를 표현하는 객체를 생성
circuit.draw(output='mpl') # circuit 회로를 matplot library 형식의 그림 화일로 출력


In [ ]:
# 그로버 오라클 함수와 디퓨저 함수를 1번 호출하여 그로버 회로를 circuit 이라는 이름의 객체로 구현
n = 3
qx = QuantumRegister(n,'x')  # n 큐비트의 양자 레지스터 객체를 qx 라는 이름으로 생성, 회로에는 'x'로 표시
qy = QuantumRegister(1,'y')  # 1 큐비트의 양자 레지스터 객체를 qy 라는 이름으로 생성, 회로에는 'y'로 표시
c = ClassicalRegister(n,'c') # 1 비트의 고전 레지스터 객체를 c 라는 이름으로 생성, 회로에는 'c'로 표시
circuit = QuantumCircuit(qy, qx, c) # qy,qx,c 를 가지는 양자 회로를 circuit 이라는 이름의 객체로 생성
circuit.x(qy)                 # qy 큐비트에 X gate -> qy 큐비트 상태가 |1> 이 됨
circuit.h(qy)                 # qy 큐비트에 H gate -> qy 큐비트 상태가 |-> 이 됨
circuit.h(qx)                 # qx 의 모든 큐비트에 H gate 적용
circuit.barrier()
for _ in range (1):           # 1번 반복
# grover_oracle_circuit() 을 호출하여 오라클 회로를 circuit 에 추가
    circuit.compose(grover_oracle_circuit(), range(n+1), inplace=True)
    circuit.barrier()
# diffuser_circuit() 을 호출하여 디퓨저 회로를 circuit 에 추가
    circuit.compose(diffuser_circuit(n), range(1,n+1), inplace=True)
    circuit.barrier()

circuit.measure(qx,c)              # qx 레지스터의 모든 큐비트들을 측정하여 c 레지스터로 읽는 회로를 구현
circuit.draw(fold=-1,output='mpl') # circuit 회로를 matplot library 형식의 그림 화일로 출력

In [ ]:
# 그로버 오라클 함수와 디퓨저 함수를 2번 호출하여 그로버 회로를 circuit 이라는 이름의 객체로 구현
n = 3
qx = QuantumRegister(n,'x')  # n 큐비트의 양자 레지스터 객체를 qx 라는 이름으로 생성, 회로에는 'x'로 표시
qy = QuantumRegister(1,'y')  # 1 큐비트의 양자 레지스터 객체를 qy 라는 이름으로 생성, 회로에는 'y'로 표시
c = ClassicalRegister(n,'c') # 1 비트의 고전 레지스터 객체를 c 라는 이름으로 생성, 회로에는 'c'로 표시
circuit = QuantumCircuit(qy, qx, c) # qy,qx,c 를 가지는 양자 회로를 circuit 이라는 이름의 객체로 생성
circuit.x(0)                 # 0번 큐비트에 X gate -> 0번 큐비트 상태가 |1> 이 됨
circuit.h(0)                 # 0번 큐비트에 H gate -> 0번 큐비트 상태가 |-> 이 됨
circuit.h(qx)                # qx 의 모든 큐비트에 H gate 적용

for _ in range (2):         # 2번 반복
# grover_oracle_circuit() 을 호출하여 오라클 회로를 circuit 에 추가
    circuit.append(grover_oracle_circuit(), range(n+1))
# diffuser_circuit() 을 호출하여 디퓨저 회로를 circuit 에 추가
    circuit.append(diffuser_circuit(n), range(1,n+1))

circuit.measure(qx,c)      # qx 레지스터의 모든 큐비트들을 측정하여 c 레지스터로 읽는 회로를 구현
circuit.draw(fold=-1,output='mpl') # circuit 회로를 matplot library 형식의 그림 화일로 출력

In [ ]:
# AerSimulator 를 이용하여 circuit 회로를 측정한 결과를 출력
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_aer import AerSimulator

aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)

isa_circuit = pm.run(circuit)
sampler = Sampler(mode=aer_sim)
job = sampler.run([isa_circuit], shots=10000)
result = job.result()
count = result[0].data.c.get_counts()

from qiskit.visualization import plot_distribution
plot_distribution(count)